# Weka Experiment

The definitive notebook.

# Preliminaries

## Imports

In [1]:
import pandas as pd
import arff
import os
import numpy as np
import aaai20
import PxW

from os.path import dirname
from aaai20.io import filename_dataset, filename_query, filename_results
from aaai20.exp import collect_results, process_outcomes, save_outcome
from sklearn.model_selection import train_test_split
from modulo.utils.encoding import query_to_code, code_to_query, encode_attribute

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

RANDOM_STATE = 42

In [2]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

## Helpers

In [3]:
def f1_weka(out, average='macro'):
    # Each label gets encoded.
    le = LabelEncoder().fit(out.values.reshape(-1,))
    
    out = out.apply(le.transform)
    f1 = f1_score(out['actual'], out['predicted'], average=average)
    return f1

## Functions

In [4]:
def fit_weka(ds,**kwargs):
    # Preliminaries
    fn_train = filename_dataset(ds, step=1, suffix='train')
    
    # Train
    clf = PxW.J48(**kwargs)
    clf.fit(fn_train, verbose=False)
    return clf

In [5]:
def predict_weka(dataset, classifier):
    result = {}
    f1_micro = []
    f1_macro = []
    
    # Get queries
    fn_qry = filename_query(dataset, suffix="default")
    q_codes = np.load(fn_qry)
    q_codes_return = q_codes.copy()
    
    for q_idx, q_code in enumerate(q_codes):
        fn = filename_dataset(dataset, step=2, suffix='q_{}'.format(str(q_idx).zfill(3)))
        
        out = clf.predict(fn, verbose=True)
        
        #print(ds)
        #print(out.head())
        
        f1_micro.append(f1_weka(out, average='micro'))
        f1_macro.append(f1_weka(out, average='macro'))
    
    result['f1_micro'] = f1_micro
    result['f1_macro'] = f1_macro
    
    return q_codes_return, result

# Actual Flow

In [6]:
datasets = ['glass',
             'credit-g',
             'ionosphere',
             'lymph',
             'vehicle',
             'iris',
             'splice',
             'sonar',
             'vowel',
             'segment',
             'zoo',
             'heart-statlog',
             'waveform-5000',
             'kr-vs-kp',
             'diabetes',
             'letter',
             'balance-scale']

MAX_DEPTH = 8
NB_ITERATIONS = 2
NB_ESTIMATORS = 10

print(len(datasets))

dataframes = {k:[] for k in datasets}

for algorithm in {"J48", "RF"}:
    for ds in datasets:
        identifier = algorithm
        
        # Get nb_atts
        fn_qry = filename_query(ds, suffix="default")
        q_codes = np.load(fn_qry)
        nb_atts = q_codes.shape[1]
        
        if NB_ESTIMATORS is None:
            NB_ESTIMATORS = nb_atts*NB_ITERATIONS
            print("""
            For dataset {}, we learn {} trees.
            """.format(ds, NB_ESTIMATORS))
        
        # do stuff
        clf = fit_weka(ds, algorithm=algorithm, max_depth=MAX_DEPTH, nb_estimators=NB_ESTIMATORS)

        q_codes, results = predict_weka(ds, clf)

        dataframes[ds] = collect_results(ds, q_codes, results, identifier=algorithm)

    df = process_outcomes(dataframes)

    fn = filename_results(exp_dname='mercs-vs-weka', exp_fname=algorithm)
    df.to_csv(fn, index=False)
   

[2019-11-12 17:06:26,872] INFO - prefect.FlowRunner | Beginning Flow run for 'fit'
[2019-11-12 17:06:26,873] INFO - prefect.FlowRunner | Starting flow run.
[2019-11-12 17:06:26,878] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-11-12 17:06:26,879] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-11-12 17:06:26,880] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...


17
java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.J48 -C 0.25 -M 2 -no-cv -t /cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/data/step-01/glass-train.arff -d tree.model


[2019-11-12 17:06:27,751] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:06:27,757] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:06:27,763] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-11-12 17:06:27,766] INFO - prefect.FlowRunner | Starting flow run.
[2019-11-12 17:06:27,772] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-11-12 17:06:27,774] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-11-12 17:06:27,775] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-11-12 17:06:28,029] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:06:28,031] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:06:28,061] INFO - prefect.FlowRunner | Beginni

java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.J48 -C 0.25 -M 2 -no-cv -t /cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/data/step-01/credit-g-train.arff -d tree.model


[2019-11-12 17:06:42,486] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:06:42,487] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:06:42,489] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-11-12 17:06:42,490] INFO - prefect.FlowRunner | Starting flow run.
[2019-11-12 17:06:42,493] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-11-12 17:06:42,494] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-11-12 17:06:42,494] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-11-12 17:06:42,725] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:06:42,726] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:06:42,735] INFO - prefect.FlowRunner | Beginni

...

[2019-11-12 17:13:01,165] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:13:01,167] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:13:01,172] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-11-12 17:13:01,173] INFO - prefect.FlowRunner | Starting flow run.
[2019-11-12 17:13:01,179] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-11-12 17:13:01,181] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-11-12 17:13:01,182] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-11-12 17:13:01,444] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:13:01,445] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:13:01,453] INFO - prefect.FlowRunner | Beginni

java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.RandomForest -I 10 -depth 8 -no-cv -t /cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/data/step-01/letter-train.arff -d tree.model


[2019-11-12 17:13:15,951] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:13:15,951] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:13:15,954] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-11-12 17:13:15,954] INFO - prefect.FlowRunner | Starting flow run.
[2019-11-12 17:13:15,957] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-11-12 17:13:15,958] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-11-12 17:13:15,958] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-11-12 17:13:16,310] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:13:16,311] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:13:16,326] INFO - prefect.FlowRunner | Beginni

java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.RandomForest -I 10 -depth 8 -no-cv -t /cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/data/step-01/balance-scale-train.arff -d tree.model


[2019-11-12 17:13:40,873] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:13:40,874] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:13:40,876] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-11-12 17:13:40,877] INFO - prefect.FlowRunner | Starting flow run.
[2019-11-12 17:13:40,880] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-11-12 17:13:40,881] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-11-12 17:13:40,882] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-11-12 17:13:41,109] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-11-12 17:13:41,110] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-11-12 17:13:41,118] INFO - prefect.FlowRunner | Beginni